<a href="https://colab.research.google.com/github/Big-Shu-Nim/NLP/blob/main/%ED%8A%B8%EB%9E%9C%EC%8A%A4%ED%8F%AC%EB%A8%B8_%EC%95%A0%ED%94%8C%EB%A6%AC%EC%BC%80%EC%9D%B4%EC%85%98_%EB%91%98%EB%9F%AC%EB%B3%B4%EA%B8%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#1.5.1 텍스트 분류


* **파이프라인**을 사용하여 로우 데이터를 미세 튜닝된 모델의 예측으로 변환하는 작업을 실시 

In [ ]:
!pip install transformers

In [ ]:
from transformers import pipeline

classifier = pipeline("text-classification")

* 위 작업이 하는 수행내역은 다음과 같다.
 * 허깅페이스 허브에서 모델 가중치를 다운로드한다.
 * 다운로드가 완료된다음에는 캐싱된 버젼을 사용한다.
* text classification의 경우 감성분석을 위해 설계된 모델,
* 그러나 다중분류 다중 레이블 분류도 가능 


In [ ]:
text ="""
Dear Amazon, last week I ordered an Optimus Prime action figure from your online store in Germany. Unfortunately, when I opened the package, I discovered to my horror that I had been sent an action figure of Megatron instead! As a lifelong enemy of the Decepticons, I hope you can understand my dilemma. To resolve the issue, I demand an exchange of Megatron for the Optimus Prime figure I ordered. Enclosed are copies of my records concerning this purchase.
 I expect to hear from you soon. Sincerely, Bumblebee."""

In [ ]:
import pandas as pd
outputs = classifier(text)
pd.DataFrame(outputs)

,label,score
0,NEGATIVE,0.901546


#1.5.2 개체명 인식
* 고객 피드백의 감성을 예측하는것은 좋은 시작점
* 그러나 피드백이 특정 제품과 서비스 중 무엇에 대한 것인지 알고 싶을때가 있음.
* NLP에서는 제품, 장소, 사람 같은 실제 객체를 개체명이라고 하고 이런 개체명을
추출하는 작업을 개체명 인식이라고 합니다.(Named entity Recognition)

# 개체명 인식 설명


당신이 하루 종일 얘기하면서 다양한 이름들을 사용할 거예요. 예를 들어, "오늘은 '서울'에서 '유나'랑 '짜장면'을 먹었어"라고 말한다면, '서울', '유나', '짜장면'은 모두 이름이죠.

여기서 '서울'은 장소의 이름이고, '유나'는 사람의 이름이며, '짜장면'은 음식의 이름입니다. 이렇게 사람, 장소, 제품 등 특정한 종류에 속하는 이름을 '개체명'이라고 해요.

그럼 이제 '개체명 인식'이 무엇인지 알아볼게요. 컴퓨터에게 이런 개체명을 찾아내는 작업을 시키는 것을 '개체명 인식'이라고 합니다. 컴퓨터가 우리가 말하는 것을 이해하려면 먼저 우리가 얘기하는 '이름들'을 알아야 해요.

예를 들어, "오늘 '유나'랑 '서울'에서 '짜장면'을 먹었어"라는 문장에서 컴퓨터가 '유나', '서울', '짜장면'이 중요한 이름임을 알아내야 한다면, 그게 바로 '개체명 인식'의 역할입니다.

이렇게 컴퓨터가 문장에서 중요한 이름들을 찾아내는 것은, 정보를 정리하거나, 검색을 할 때 정말 중요하답니다. 이제 '개체명 인식'이 무엇인지 이해가 되셨을 거예요!

In [ ]:
ner_tagger = pipeline('ner', aggregation_strategy='simple')
outputs = ner_tagger(text)
pd.DataFrame(outputs)

No model was supplied, defaulted to dbmdz/bert-large-cased-finetuned-conll03-english and revision f2482bf (https://huggingface.co/dbmdz/bert-large-cased-finetuned-conll03-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


,entity_group,score,word,start,end
0,ORG,0.879010,Amazon,6,12
1,MISC,0.990859,Optimus Prime,37,50
2,LOC,0.999755,Germany,91,98
3,MISC,0.556571,Mega,209,213
4,PER,0.590255,##tron,213,217
5,ORG,0.669692,Decept,254,260
6,MISC,0.498349,##icons,260,265
7,MISC,0.775362,Megatron,351,359
8,MISC,0.987854,Optimus Prime,368,381
9,PER,0.812096,Bumblebee,504,513


In [ ]:
# 기호는 해당모델의 토크나이저가 생성한것이다.
# 단어가 두단어여도 개체명을 확신한다면 두 단어로 구성된다.
# 그러나 개체명을 확실할수 없다면 개별 토큰으로 분류해서 점수가 매겨진다.

#1.5.3 질문답변

In [ ]:
reader = pipeline("question-answering")
question = "what does the customer want?"
outputs = reader(question=question, context=text)
pd.DataFrame([outputs])

No model was supplied, defaulted to distilbert-base-cased-distilled-squad and revision 626af31 (https://huggingface.co/distilbert-base-cased-distilled-squad).
Using a pipeline without specifying a model name and revision in production is not recommended.


,score,start,end,answer
0,0.642405,336,359,an exchange of Megatron


#1.5.4 요약

* 긴 텍스트를 입력으로 받고 간단하게 출력하는 작업
* 논리적이여야함으로 더 어려운 과제

In [ ]:
summarizer = pipeline('summarization')
outputs = summarizer(text,min_length=40, max_length=46, clean_up_tokenization_spaces=True)
print(outputs[0]['summary_text'])


No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.


 Bumblebee orders an Optimus Prime action figure from an online store in Germany. When he opened the package, he discovered to his horror that he had been sent an action figure of Megatron instead. As a lifelong


max_length와 clean_up_tokenization_spaces는 Hugging Face의 transformers library에서 제공하는 파라미터들입니다.

max_length: 이것은 생성되는 요약문의 최대 길이를 정의합니다. 예를 들어, max_length=56이라고 설정하면, 요약문의 길이는 56 토큰을 넘지 않습니다. 토큰은 주로 단어나 문자를 의미하는데, 언어와 토큰화 방식에 따라 약간씩 다릅니다. 이 값을 너무 작게 설정하면 요약문이 너무 짧아질 수 있고, 너무 크게 설정하면 요약문이 원문과 거의 차이가 없어질 수 있습니다.

clean_up_tokenization_spaces: 토큰화 과정에서 생성된 여분의 공백을 제거할지 결정하는 파라미터입니다. True로 설정하면, 모델이 생성한 출력에서 토큰화 과정에서 생긴 여분의 공백을 제거합니다.

따라서 위 코드는, 주어진 text를 최대 56 토큰으로 요약하고, 토큰화 과정에서 생긴 여분의 공백을 제거하여 출력하라는 것을 의미합니다. 이후 print(outputs[0]['summary_text'])를 통해 첫 번째 요약문만 출력하고 있습니다.

#1.5.5 번역

In [ ]:
translator = pipeline('translation_en_to_ko',
                      model=?)
outputs = translator(text, clean_up_tokenization_spaces=True, min_lenth=100)
print(outpus[0]['translation_text'])


In [ ]:
!pip install sentencepiece


In [ ]:
from transformers import pipeline

translator = pipeline('translation_en_to_ko', model="facebook/m2m100_418M")
outputs = translator(text, clean_up_tokenization_spaces=True, max_length=400)
print(outputs[0]['translation_text'])



사랑하는 아마존, 지난 주에 나는 독일에서 온라인 상점에서 Optimus Prime 액션 숫자를 주문했습니다. 불행히도, 패키지를 열었을 때, 나는 나의 공포에 메가트론 액션 숫자를 보냈다는 것을 발견했습니다! Decepticons의 평생 적으로서, 나는 당신이 나의 딜레마를 이해할 수 있기를 바랍니다. 문제를 해결하기 위해, 나는 내가 주문 한 Optimus Prime 숫자에 대한 메가트론 교환을 요구합니다.


In [ ]:
generator = pipeline('text-generation')
response = "Dear Bumblebee, I am sorry to hear that your dorer was mixed up"
prompt = text + '\n\nCustomer service response:\n'+ response
outputs = generator(prompt, max_length=200)
print(outputs[0]['generated_text'])

No model was supplied, defaulted to gpt2 and revision 6c0e608 (https://huggingface.co/gpt2).
Using a pipeline without specifying a model name and revision in production is not recommended.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



Dear Amazon, last week I ordered an Optimus Prime action figure from your online store in Germany. Unfortunately, when I opened the package, I discovered to my horror that I had been sent an action figure of Megatron instead! As a lifelong enemy of the Decepticons, I hope you can understand my dilemma. To resolve the issue, I demand an exchange of Megatron for the Optimus Prime figure I ordered. Enclosed are copies of my records concerning this purchase.
 I expect to hear from you soon. Sincerely, Bumblebee.

Customer service response:
Dear Bumblebee, I am sorry to hear that your dorer was mixed up. If you were not correct when your request was made for the Optimus Prime figure, you will receive an apology from Bumblebee. I will notify you immediately to check the status of my order.

Your customer service is essential to fulfilling your orders. Your review will be shared by other customers to inform them of our


# 1.6.4 허깅페이스 엑셀러레이트

"허깅페이스 엑셀러레이트"는 딥러닝 모델 훈련을 쉽고 효과적으로 실행하기 위한 도구입니다. 이 도구는 훈련 프로세스를 추상화하여 여러 가지 다른 하드웨어 환경에서 동작할 수 있게 돕습니다.

"사용자의 로직을 처리하는 일반적인 훈련 루프" : 이것은 일반적으로 입력 데이터를 모델에 전달하고, 손실을 계산하고, 그래디언트를 업데이트하고, 이 과정을 반복하는 머신러닝의 기본적인 훈련 프로세스를 의미합니다.

"훈련 인프라에 필요한 추상화층 을 추가" : 여기서 "추상화층"이란 특정 구현 세부사항을 숨기고 사용자에게 단순한 인터페이스를 제공하는 것을 말합니다. 예를 들어, 엑셀러레이트는 여러 GPU 또는 여러 컴퓨터, 혹은 TPU와 같은 다양한 하드웨어 환경에서 동작하는 로직을 추가하지 않아도 훈련이 가능하게 합니다. 즉, 엑셀러레이트가 이런 세부사항을 대신 처리해주므로 사용자는 자신의 모델과 데이터에 집중할 수 있습니다.

"인프라 전환을 단순화시키고 워크플로를 가속화" : 엑셀러레이트를 사용하면 한 가지 하드웨어 설정에서 다른 하드웨어 설정으로 쉽게 전환할 수 있습니다. 예를 들어, 단일 GPU에서 훈련을 시작한 후, 여러 GPU를 사용하도록 쉽게 전환할 수 있습니다. 이는 여러 하드웨어에서 동작하는 복잡한 코드를 작성하는 수고를 덜어주고, 따라서 워크플로를 더 빠르게 진행할 수 있게 돕습니다.

이렇게 볼 때, "허깅페이스 엑셀러레이트"는 여러 하드웨어 환경에서 훈련을 쉽게 하도록 도와주는 도구라고 이해하시면 좋을 것 같습니다.